**Sentiment Analysis for Turkish**

Merhaba. Bu çalışmada, Kaggle'dan aldığım "turkish_cyberbullying" veri setinde, hazır bulunan duygu analizi fonksiyonunu kullanarak sonuç bulmaya çalışacağım. Bu konuda izlediğim yol şu şekildedir:

    - Gerekli kütüphane ve verinin indirilmesi
    - Veri ön işlemesi (preprocessing) yapılması
    - Tweet'lerin İngilizceye çevirilmesi
    - Textblob kütüphanesi ile cümlelerin duygu analizinin bulunması
    - Veri setinde verilmiş olan duygu analizi ile karşılaştırılması

In [3]:
# kütüphanelerin indirilmesi
# - *- coding: utf- 8 - *-
import pandas as pd
import trnlp

In [4]:
# trnlp türkçe dil işleme için hazırlanmış olan bir kütüphanedir.
# https://github.com/brolin59/PYTHON-TURKCE-DOGAL-DIL-ISLEME-TURKISH-NLP
help(trnlp)

Help on package trnlp:

NAME
    trnlp

PACKAGE CONTENTS
    Asciidecoder (package)
    Morphological (package)
    Statistic (package)
    Tokenization (package)
    file_processing
    helpers
    tr_suffix
    word_processing

FUNCTIONS
    deascii(str_grp: str)
    
    find_stems(word: str)
    
    find_suffix(word: str)
    
    ntow(number: str)
    
    unknown_words(str_grp: str)
    
    view_statistic(str_grp: str)
    
    wton(word: str)

DATA
    clean_quiet = re.compile('[^aâeêıîioôöuûü]')
    current_file = 'C:/Users/gmdemirci/trnlp'
    lower_quiet = 'bcçdfgğhjklmnprsştvyzqwx'
    lower_vowel = 'aâeêıîioôöuûü'

FILE
    c:\users\gmdemirci\trnlp\__init__.py




Verisetinde iki adet başlık bulunuyor. Twitter'da yazılmış olan tweetler (*message*) ve bu tweetin siber zorbalık olup olmadığı (*cyberbullying*). Siber zorbalık olan tweetler 1, olmayanlar 0 olarak alınmış. 

Bununla birlikte ben de, bulacağım duygu analizinde sıfırdan küçük olanları (negative sentiment) "0" olarak, sıfır ve sıfırdan büyük olanları (positive sentiment) "1" olarak atayacağım. Böylelikle çeviri ile yapılan duygu analizinde hata matrisini görmüş olacağım.

In [48]:
data = pd.read_csv(r"C:\Users\gmdemirci\Desktop\GMD\DATA SCIENCE\PYTHON\DATA SETS\TURKISH CYBERBULLYING\turkish_cyberbullying.csv")
data.head()

,message,cyberbullying
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0
2,oo süpersin azıcık bize de bulaşsa,0
3,bende biliyorum benden bı bok olmicak,1
4,nerdesin len tirrek,1


In [50]:
data.info() # verisetimizde 3000 adet tweet bulunmaktadır.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 2 columns):
message          3001 non-null object
cyberbullying    3001 non-null int64
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


** Metin ön işlemlerimi asıl verinin üzerinde yapmak istemediğimden dolayı, yeni bir sütun oluşturarak message sütununu kopyalayacağım. Veri ön işlememi bu sütun ile yapacağim.

In [51]:
data["cleaned_text"] = data["message"]
data.head()

,message,cyberbullying,cleaned_text
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0,bir ateist olarak bu resmi gördükçe gözyaşları...
2,oo süpersin azıcık bize de bulaşsa,0,oo süpersin azıcık bize de bulaşsa
3,bende biliyorum benden bı bok olmicak,1,bende biliyorum benden bı bok olmicak
4,nerdesin len tirrek,1,nerdesin len tirrek


In [52]:
""" DATA PREPROCESSING"""
# 1) tüm kelimeleri küçük harf yapma
data.cleaned_text = [each.lower() for each in data.cleaned_text]
data.head()

,message,cyberbullying,cleaned_text
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0,bir ateist olarak bu resmi gördükçe gözyaşları...
2,oo süpersin azıcık bize de bulaşsa,0,oo süpersin azıcık bize de bulaşsa
3,bende biliyorum benden bı bok olmicak,1,bende biliyorum benden bı bok olmicak
4,nerdesin len tirrek,1,nerdesin len tirrek


In [53]:
# 2) noktalama işaretlerini silme
# re kütüphanesi ile silme, yerine başka kelime getirme gibi cümle düzenlemesi yapılabilmektedir
import re

data.cleaned_text = [re.sub(r'[!”"#$%&’()*+,-./:;<=>?@[\]''^_`{|}~]','',each) 
                     if type(each) == str else each for each in data.cleaned_text]
data.head()

,message,cyberbullying,cleaned_text
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0,bir ateist olarak bu resmi gördükçe gözyaşları...
2,oo süpersin azıcık bize de bulaşsa,0,oo süpersin azıcık bize de bulaşsa
3,bende biliyorum benden bı bok olmicak,1,bende biliyorum benden bı bok olmicak
4,nerdesin len tirrek,1,nerdesin len tirrek


In [54]:
# 3) sayıları silme
data.cleaned_text = [re.sub(r'\d+','',each) for each in data.cleaned_text]
data.head()

,message,cyberbullying,cleaned_text
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0,bir ateist olarak bu resmi gördükçe gözyaşları...
2,oo süpersin azıcık bize de bulaşsa,0,oo süpersin azıcık bize de bulaşsa
3,bende biliyorum benden bı bok olmicak,1,bende biliyorum benden bı bok olmicak
4,nerdesin len tirrek,1,nerdesin len tirrek


*** Türkçe nlp olan "trnlp" paketinin *deascii* metodu kullanılarak türkçe karakter ile yazılmamış kelimeleri tamamlayacağız. "ozur" yerine "özür" gibi.

In [55]:
# 4) Türkçe kelime normalizasyonu
data.cleaned_text = [trnlp.deascii(each) for each in data.cleaned_text]
data.head()

,message,cyberbullying,cleaned_text
0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,0,rabbim kalan ömrünü geçen ömründen hayırlı eyl...
1,bir ateist olarak bu resmi gördükçe gözyaşları...,0,bir ateist olarak bu resmi gördükçe gözyaşları...
2,oo süpersin azıcık bize de bulaşsa,0,oo süpersin azıcık bize de bulaşsa
3,bende biliyorum benden bı bok olmicak,1,bende biliyorum benden bi bok olmıçak
4,nerdesin len tirrek,1,nerdesin len tirrek


In [56]:
# # 4) metin tokenization
# tokenized_text = data["cleaned_text"].apply(lambda x: trnlp.token(x))
# tokenized_text.head()

Bu çalışmadaki duygu analizi fonksiyon bazlıdır. Makine öğrenmesi bulunmamaktadır. Textblob kütüphanesindeki önceden yazılmış metod ile ilerleyeceğim. Ancak textblob duygu analizinde Türkçe duygu analizi bulunmamaktadır. Bundan dolayı, verisetindeki cümleleri "Google Translate API" ile İngilizce'ye çevirerek duygu analizi yapacağım.

Böylelikle, Türkçe'de henüz tam oturtulmamış duygu analizi fonksiyon eksikliğini bu şekilde tamamlayıp tamamlayamadığımızı görmüş olacağım. 

In [101]:
# 5) Cümleleri İngilizce diline çevirme
    # a. Çeviri için dataframe'i Listeye çevirme
translate_list = list(data.cleaned_text[0:3000])
translate_list[0:10]

['rabbim kalan ömrünü geçen ömründen hayırlı eylesin',
 'bir ateist olarak bu resmi gördükçe gözyaşlarıma mani olamıyorum',
 'oo süpersin azıcık bize de bulaşsa',
 'bende biliyorum benden bi bok olmıçak',
 'nerdesin len tirrek',
 'doğruyusöyleyince kadro verince adalet yerini bulacak',
 'öğrenciler hocalarına memleket duruşuna hayran ',
 'günaydın hala yatakta olanlar eminim elinizde telefon aletinizde tad gibidir hadi sanalda sabah keyfi yapalım',
 'artist pezevenk takımı yaktı rezildi',
 've o narinlikte ve sevimlilikte cüce öylesine acımasızca zibh eğliyip']

In [102]:
# 5) Cümleleri İngilizce diline çevirme
    # b. google.cloud ile çeviriyi yapma  
# - *- coding: utf- 8 - *-

from google.cloud import translate
translate_client = translate.Client()
translation = [ translate_client.translate(each,target_language="en") for each in translate_list]

* ** 3000 adet satır 13 dakika içerisinde çevirisi yapılmıştır. **

In [103]:
# çevrilen dil, çevirilen cümle ve çevirisi olarak çıktı verdi.
translation = pd.DataFrame(translation)
translation.head()

,detectedSourceLanguage,input,translatedText
0,tr,rabbim kalan ömrünü geçen ömründen hayırlı eyl...,rabbim remaining lifetime
1,tr,bir ateist olarak bu resmi gördükçe gözyaşları...,as an atheist I can&#39;t stop my tears as I s...
2,tr,oo süpersin azıcık bize de bulaşsa,If you&#39;re a super-junkie
3,tr,bende biliyorum benden bi bok olmıçak,I know I&#39;m a fucking shit.
4,tr,nerdesin len tirrek,Where are you?


** Benim ihtiyacım olan, sadece İngilizce cümle. Bundan dolayı diğer sütunları listemden çıkarıyorum.

In [104]:
translation.drop("detectedSourceLanguage", axis=1,inplace=True)
translation.drop("input", axis=1,inplace=True)
translation.head()

,translatedText
0,rabbim remaining lifetime
1,as an atheist I can&#39;t stop my tears as I s...
2,If you&#39;re a super-junkie
3,I know I&#39;m a fucking shit.
4,Where are you?


** Çeviri sırasında utf-8 sıkıntısı yaşandı. kesme işareti (') (&#39 ;) olarak gösterilmektedir. Bunu düzeltmek adına, tekrardan re kütüphanesini kullanıyorum.

In [105]:
translation.translatedText = [re.sub("&#39;","'",each) 
                     if type(each) == str else each for each in translation.translatedText]
translation.head()

,translatedText
0,rabbim remaining lifetime
1,as an atheist I can't stop my tears as I see t...
2,If you're a super-junkie
3,I know I'm a fucking shit.
4,Where are you?


** Duygu analizi için cümleler liste ile verilmeli. Bundan dolayı, Dataframe'mi listeye çevirerek analizi yapıyorum.

In [106]:
from textblob import TextBlob
sentiment = translation['translatedText'].values.tolist()
sentiment = [TextBlob(each).sentiment.polarity for each in sentiment]
sentiment[0:10]

[0.0,
 0.0,
 0.0,
 -0.2,
 0.0,
 0.0,
 0.0,
 0.5333333333333333,
 0.0,
 -0.21666666666666665]

Şimdi, veriseti ile gelmiş olan cyberbullying ile bizim bulduğumuz duygu analizini aynı tabloya koyacağım, ancak öncesinde duygu analizinde çıkan sayıları 1 ve 0 olarak ayarlayalım. 

** Negatif olanlar 0, sıfır ve pozitif olanları 1 olarak atıyoruz.

In [108]:
duygu = []

for row in sentiment:
    if row < 0:
        # Append a 1
        duygu.append(1)
    else:
        duygu.append(0)

translated = pd.DataFrame(duygu)    
translated["cyberbullying"] = data.cyberbullying[0:3000]
translated.head()

,0,cyberbullying
0,0,0
1,0,0
2,0,0
3,1,1
4,0,1


In [113]:
# sütun isimlerini değiştiriyorum
translated.columns = [["duygu_analizi","cyberbullying"]]
translated.head()

,duygu_analizi,cyberbullying
0,0,0
1,0,0
2,0,0
3,1,1
4,0,1


** 3000 satır tweet'imizin çevirisi yapılıp, duygu analizine girmiştir. 

In [110]:
translated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
(duygu_analizi,)    3000 non-null int64
(cyberbullying,)    3000 non-null int64
dtypes: int64(2)
memory usage: 47.0 KB


** 3000 veri için doğruluk testi yaparak, bu işlemin uygulanabilir olup olmadığına bakalım:

In [115]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true=translated.cyberbullying, y_pred=translated.duygu_analizi)
print("Accuracy: % {:3f}".format(accuracy*100)) 

Accuracy: % 54.666667


%54,67 bizim çalışmamız için çok düşük bence. Doğruluk yüzdesinin bu kadar düşük olmasının sebeplerinden en önemlisi, verinin ön işlemesi için kullandığım **trnlp** kütüphanesinin yeterli olmaması diye düşünüyorum. Cümlelerdeki kelimeleri doğru normalizasyon yapamadığımız için çevirisi de tamamen yanlış oldu haliyle doğruluğu tutmadı. 

Türkçe için Textblob gibi duygu analizi fonksiyonu bulunmuyor şu ana kadar araştırdığım kadarıyla. Genelde duygu analizi için denetimli makine öğrenmesi kullanılıyor. Bunun için aldığım veri setlerinden bir kısmını train olarak kendim puan vermeliyim, ancak bu da yeterli doğruluk vermeyecektir sanırım.

Bag of words, Word2ve-doc2vec metodlarını kullanarak kelime ağırlığına göre analiz yapılıyor. Bunları deneyerek en verimlisini bulmayı hedefliyorum. Geri dönüş ve önerilerinizi dört gözle bekliyorum. Teşekkürler.

                                                                                                                      Gözde